![](/Volumes/datamaster/bronze/images/Gemini_Generated_Image_yoay4cyoay4cyoay.png)

In [0]:
%sql
USE CATALOG datamaster;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS datamaster.bronze;

In [0]:
%sql

CREATE OR REPLACE TABLE datamaster.bronze.test (id int, name string, age int);
INSERT INTO datamaster.bronze.test VALUES (1,'David',34)

In [0]:
%sql
SELECT * FROM datamaster.bronze.test

#Functions

In [0]:
%sql
USE CATALOG datamaster;
USE SCHEMA bronze;

CREATE OR REPLACE TABLE customer_reviews (
    customer_id INT,
    first_name  STRING,
    last_name   STRING,
    review      STRING
);

INSERT INTO customer_reviews VALUES
(1,'John','Smith','Great product!'),
(2,'Jane','Doe','Not so great'),
(3,'Bob','Jones','Excellent'),
(4,'Alice','Williams','Very good'),
(5,'Charlie','Brown','Not bad'),
(6,'David','Lee','Very poor');



In [0]:
SELECT * FROM datamaster.bronze.customer_reviews;

In [0]:
CREATE OR REPLACE FUNCTION datamaster.bronze.full_name_udf(first_name string, last_name string)
RETURNS STRING
RETURN first_name || ' ' || last_name

In [0]:
SELECT customer_id, datamaster.bronze.full_name_udf(first_name, last_name) AS full_name, review
FROM datamaster.bronze.customer_reviews;

In [0]:
CREATE OR REPLACE FUNCTION datamaster.bronze.sentiment_analysis(review STRING)
RETURNS STRING
RETURN
    CASE
        WHEN review LIKE '%Great%' OR review LIKE '%Excellent%' OR review LIKE '%good%' THEN 'positive'
        WHEN review LIKE '%not%' OR review LIKE '%poor%' THEN 'negative'
        ELSE 'neutral'
    END;

In [0]:
SELECT 
customer_id,
datamaster.bronze.full_name_udf(first_name, last_name) AS full_name,
review,
datamaster.bronze.sentiment_analysis (review) AS sentiments
FROM datamaster.bronze.customer_reviews;

#Views

Virtual Tables
1. Standard view : persisted (saved)
2. Temp View / Global Temp View

In [0]:
CREATE SCHEMA IF NOT EXISTS datamaster.silver;
CREATE SCHEMA IF NOT EXISTS datamaster.gold;

In [0]:
CREATE OR REPLACE VIEW datamaster.silver.customer_sentiment AS
SELECT 
customer_id,
datamaster.bronze.full_name_udf(first_name, last_name) AS full_name,
review,
datamaster.bronze.sentiment_analysis (review) AS sentiments
FROM datamaster.bronze.customer_reviews;

In [0]:
CREATE OR REPLACE VIEW datamaster.gold.customer_sentiment AS
SELECT
    datamaster.bronze.sentiment_analysis(review) AS sentiment,
    COUNT(*) AS count
FROM datamaster.bronze.customer_reviews
GROUP BY sentiment_analysis(review)


In [0]:
SELECT * FROM datamaster.gold.customer_sentiment

Databricks visualization. Run in Databricks to view.